# Intro to Decision Trees

### Learning Objectives

- Understand the intuition behind decision trees.
- Calculate Gini.
- Describe how decision trees use Gini to make decisions.
- Fit, generate predictions from, and evaluate decision tree models.
- Interpret and tune `criterion`, `max_depth`, `min_samples_split`.

## What will we get for dinner?

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   Indian  |      Rainy     |   Weekday  |
|   Sushi   |      Sunny     |   Weekday  |
|   Indian  |      Rainy     |   Weekend  |
|  Mexican  |      Sunny     |   Weekend  |
|   Indian  |      Rainy     |   Weekday  |
|  Mexican  |      Sunny     |   Weekend  |

---

<details><summary>It's a rainy day. Based on our past orders, what do you think we'll order?</summary>

- Indian food.
- In 100% of past cases where the weather is rainy, we've eaten Indian food!

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   Indian  |      Rainy     |   Weekday  |
|   Indian  |      Rainy     |   Weekend  |
|   Indian  |      Rainy     |   Weekday  |

</details>

---

<details><summary>It's a sunny day. Based on our past orders, what do you think we'll order?</summary>

- Either Sushi or Mexican food... but we can't say with certainty whether we'd eat sushi or Mexican food.
- Based on our past orders, we eat sushi on 1/3 of sunny days and we eat Mexican food on 2/3 of sunny days.
- If I **had** to make a guess here, I'd probably predict Mexican food, but we may want to use additional information to be certain.

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   Sushi   |      Sunny     |   Weekday  |
|  Mexican  |      Sunny     |   Weekend  |
|  Mexican  |      Sunny     |   Weekend  |

</details>

---

<details><summary>It's a sunny day that also happens to be a weekend. Based on our past orders, what do you think we'll order?</summary>

- Mexican food.
- In 100% of past cases where the weather is sunny and where it's a weekend, we've eaten Mexican food!

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|  Mexican  |      Sunny     |   Weekend  |
|  Mexican  |      Sunny     |   Weekend  |

</details>

# Decision Trees: Overview

A decision tree:
- takes a dataset consisting of $X$ and $Y$ data, 
- finds rules based on our $X$ data that partitions (splits) our data into smaller datasets such that
- by the bottom of the tree, the values $Y$ in each "leaf node" are as "pure" as possible.

We frequently see decision trees represented by a graph.

- (This image was created using [Draw.io](https://www.draw.io/).)

Decision trees look like upside down trees. 
- What we see on top is known as the "root node," through which all of our observations are passed.
- At each internal split, our dataset is partitioned.
- At each of the "leaf nodes" (colored orange), we contain a subset of records that are as pure as possible.
    - In this food example, each leaf node is perfectly pure. Once we get to a leaf node, every observation in that leaf node has the exact same value of $Y$!
    - There are ways to quantify the idea of "purity" here... we'll come back to this later.

Decision trees are also called "**Classification and Regression Trees**," sometimes abbreviated "**CART**."
- [DecisionTreeClassifier Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
- [DecisionTreeRegressor Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

## 20 Questions

If you aren't familiar with the game [20 Questions](https://en.wikipedia.org/wiki/Twenty_Questions), it's a game with two players (or teams). 
- Player A thinks of an item but doesn't say what the item is.
- Player B then attempts to guess what the item is by asking a series of 20 questions with a yes or no answer.
- If player B correctly guesses the item, then player B wins!
- If player B does not correctly guess the item, then player A wins!

Let's play a quick game of 20 Questions to get a feel for it.

---

Decision trees operate in a fashion that's pretty similar to 20 questions.
- Decisions are made in a sequential fashion. Once you know a piece of information, you use that piece of information when asking future questions.
    - Example: If you know that the item you're trying to guess is a person, then you can use that information to ask better subsequent questions.
- It's possible to get lucky by making very specific guesses early, but it's pretty unlikely that this is a winning strategy.
    - Example: If you asked, "Is it an airplane? Is it a boat? Is it a car?" as your first three questions, it's not very likely that you'll win the game.

When fitting a decision tree, we're effectively getting our computer to play a game of 20 Questions. We give the computer some data and it figures out the best $X$ variable to split on at the right time.
- Above, our "what food should we order?" decision tree first asked what the weather was, **then** asked whether it was a weekday or weekend.
- If we had asked "is it a weekday or weekend" first, we'd have ended up with a slightly more complicated decision tree.

Just like with all of our models, in order for the computer to learn which $X$ variable to split on and when, the computer needs a loss function to quantify how good a particular split is. This is where the idea of **purity** comes into play.

## Purity in Decision Trees

When quantifying how "pure" a node is, we want to see what the distribution of $Y$ is in each node, then summarize this distribution with a number.

<img src="./images/order_food_dt.png" alt="order_food" width="750"/>

- For continuous $Y$ (i.e. using a decision tree to predict income), the default option is mean squared error.
    - When the decision tree is figuring out which split to make at a given node, it picks the split that minimizes the MSE at that step.
    
    
- For discrete $Y$, the default option is the Gini impurity. (This is not quite the same thing as the [Gini coefficient](https://en.wikipedia.org/wiki/Gini_coefficient).)

$$
\begin{eqnarray*}
\text{Gini} &=& 1 - \sum_{i=1}^{classes} P(\text{class i})^2
\end{eqnarray*}
$$

- We use Gini to measure how pure a split is.
- Gini impurity ranges from 0 to 0.5, where:
    - 0 means "most pure."
    - 0.5 means "least pure."

In [1]:
y = ['Indian', 'Sushi', 'Indian', 'Mexican', 'Indian', 'Mexican']

In [2]:
def gini(y):
    
    gini_sum = []
    for class_i in set(y):
        prob = (y.count(class_i) / len(y))
        gini_sum.append(prob ** 2)

    return 1 - sum(gini_sum)

In [3]:
gini(y)

0.6111111111111112

In [4]:
gini(['Indian', 'Indian', 'Indian', 'Indian', 'Indian'])

0.0

In [5]:
gini(['Indian', 'Indian', 'Mexican', 'Mexican', 'Mexican', 'Mexican'])

0.4444444444444444

In [6]:
gini(['Indian', 'Indian', 'Mexican', 'Mexican', 'Indian', 'Mexican'])

0.5

In [7]:
gini(['Indian', 'Indian', 'Mexican', 'Mexican', 'Sushi', 'Sushi'])

0.6666666666666667

In [8]:
gini(['bacon', 'bacon', 'bacon', 'broc', 'broc'])

0.48

In [9]:
gini(['bacon', 'bacon', 'bacon', 'broc', 'broc']) - (2/5 * gini(['bacon', 'bacon']) + 3/5 * gini(['broc', 'broc', 'bacon']))

0.21333333333333332

In [10]:
gini(['bacon', 'bacon', 'bacon', 'broc', 'broc']) - (1/5 * gini(['broc']) + 4/5 * gini(['bacon', 'bacon', 'bacon', 'broc']))

0.17999999999999994

<details><summary>Under what circumstances should our Gini be 0?</summary>
- When all of our observations are identical.
</details>

<details><summary>Under what circumstances should our Gini be 0.5?</summary>
- When we have exactly two outcomes that are equally represented.
</details>

### So how does a decision tree use Gini to decide which variable to split on?

- At any node, consider the subset of records that exist at that node.
- Iterate through each variable that could potentially split the data.
- Calculate the Gini impurity for every split.
- Select the variable that causes the greatest decrease in Gini impurity from the parent node to the child node.

One consequence of this is that a decision tree is fit using a **greedy** algorithm. Simply put, a decision tree makes the best short-term decision by optimizing at each node individually. This might mean that our tree isn't optimal in the long run!

In [11]:
import pandas as pd

df = pd.DataFrame([
    ['DC', 'Batman', True, True, True],
    ['DC', 'Superman', False, True, True],
    ['DC', 'Flash', False, True, False],
    ['DC', 'Wonder Woman', False, True, True],
    ['DC', 'Aquaman', False, False, True],
    ['Marvel', 'Iron Man', True, False, True],
    ['Marvel', 'Captain America', False, False, False],
    ['Marvel', 'Black Widow', True, False, False],
    ['Marvel', 'Dare Devil', False, True, False],
    ['Marvel', 'Spider-Man', False, True, True]
], columns=['universe', 'name', 'tech', 'dead_parents', 'ends_with_man_or_woman'])

In [12]:
df = df.drop('name', axis=1)

In [13]:
target = 'universe'

parent = df[target].tolist()
total = len(parent)

print( total )
print( gini(parent) )

10
0.5


In [14]:
column = 'tech'

In [15]:
child_1 = df[df[column] == True][target].tolist()
len(child_1)
gini(child_1)

0.4444444444444444

In [16]:
child_2 = df[df[column] != True][target].tolist()
len(child_2)
gini(child_2)

0.48979591836734704

In [17]:
df.head(1)

,universe,tech,dead_parents,ends_with_man_or_woman
0,DC,True,True,True


In [18]:
def calculate_gini(column, target='universe'):
    parent = df[target].tolist()
    total = len(parent)
    gini_parent = gini(parent)
    
    child_1 = df[df[column] == True][target].tolist()
    total_child_1 = len(child_1)
    gini_child_1 = gini(child_1)
    
    child_2 = df[df[column] != True][target].tolist()
    total_child_2 = len(child_2)
    gini_child_2 = gini(child_2)
    
    ig = gini_parent - (
        gini_child_1 * total_child_1/total + 
        gini_child_2 * total_child_2/total
    )
    return ig

In [19]:
for column in ['tech', 'dead_parents', 'ends_with_man_or_woman']:
    print(column, '->', calculate_gini(column))

tech -> 0.023809523809523725
dead_parents -> 0.08333333333333337
ends_with_man_or_woman -> 0.08333333333333337


## Building a Decision Tree

In [20]:
import pandas as pd

# Read in Titanic data.
titanic = pd.read_csv('data/titanic_clean.csv')

target = 'Survived'
y = titanic[target]
X = titanic.drop(target, axis=1)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [22]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder

In [23]:
X.head(1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.25,S


In [24]:
mapper = DataFrameMapper([
    ('Pclass', None), 
    ('Sex', LabelEncoder()),
    ('Age', None), 
    ('Fare', None),
    ('Embarked', LabelEncoder())
])

In [25]:
Z_train = mapper.fit_transform(X_train)
Z_test = mapper.transform(X_test)

In [26]:
from sklearn.tree import DecisionTreeClassifier

In [27]:
model = DecisionTreeClassifier(random_state = 42)

In [28]:
model.fit(Z_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [29]:
print(f'Score on training set: {model.score(Z_train, y_train)}')
print(f'Score on testing set: {model.score(Z_test, y_test)}')

Score on training set: 0.9919678714859438
Score on testing set: 0.6915887850467289


<details><summary>What conclusion would you make here?</summary>

- Our model is **very** overfit to the data.
</details>

When fitting a decision tree, your model will always grow until it nearly perfectly predicts every observation!
- This is like playing a game of 20 questions, but instead calling it "Infinite Questions." You're always going to be able to win!

<details><summary>Intuitively, what might you try to do to solve this problem?</summary>
    
- As with all models, try to gather more data.
- As with all models, remove some features.
- Is there a way for us to stop our model from growing? (Yes!)
</details>

### Hyperparameters of Decision Trees
There are two hyperparameters of decision trees that we may commonly tune in order to prevent overfitting.

- `max_depth`: The maximum depth of the tree.
    - By default, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
    - In the 20 questions analogy, this is like how many questions we can ask?
- `min_samples_split`: The minimum number of samples required to split an internal node.
    - By default, minimum number of samples required to split is 2. That is, if there are two observations in a node, we can split it if maximum purity hasn't been reached!
- `min_samples_leaf`: The minimum number of samples required to be in a leaf node (a terminal node at the end of the tree).
    - 

[Source: Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

In [30]:
dt = DecisionTreeClassifier(max_depth = 5,
                            min_samples_split = 7,
                            min_samples_leaf = 3,
                            random_state = 42)

In [31]:
dt.fit(Z_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [32]:
print(f'Score on training set: {dt.score(Z_train, y_train)}')
print(f'Score on testing set: {dt.score(Z_test, y_test)}')

Score on training set: 0.8674698795180723
Score on testing set: 0.7757009345794392


#### Let's GridSearch to try to find the best tree.

- Check [3, 5, 7, 10] for `max_depth`.
- Check [5, 10, 15, 20] for `min_samples_split`.
- Check [2, 3, 4, 5, 6, 7] for `min_samples_leaf`.
- Run 5-fold cross-validation.

<details><summary>How many models are being fit here?</summary>

- 4 * 4 * 6 * 5 = 480 models.
</details>

In [33]:
from sklearn.model_selection import GridSearchCV

In [34]:
grid = GridSearchCV(estimator=DecisionTreeClassifier(),
                    param_grid={'max_depth': [3, 5, 7, 10],
                                'min_samples_split': [5, 10, 15, 20],
                                'min_samples_leaf': [2, 3, 4, 5, 6, 7]},
                    cv=5,
                    verbose = 1,
                    return_train_score = True)

In [35]:
import time

t0 = time.time()

grid.fit(Z_train, y_train)

print(time.time() - t0)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8958678245544434


[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:    0.9s finished
/Users/max/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [36]:
grid.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=15,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [37]:
grid.best_params_

{'max_depth': 7, 'min_samples_leaf': 6, 'min_samples_split': 15}

In [38]:
grid.best_score_

0.8152610441767069

In [39]:
dt = DecisionTreeClassifier(max_depth = 7,
                            min_samples_split = 15,
                            min_samples_leaf = 6,
                            random_state = 42)

dt.fit(Z_train, y_train)

print(f'Score on training set: {dt.score(Z_train, y_train)}')
print(f'Score on testing set: {dt.score(Z_test, y_test)}')

Score on training set: 0.8734939759036144
Score on testing set: 0.7383177570093458


## What's so great about decision trees?


### 1. We don't have to scale our data.
Much in the same way that decision trees don't need much preprocessing in order to work, the base assumptions around how data is used and estimated, don't rely on scale to be effective.

### 2. Decision trees don't care about how your data is distributed.

Is your data heavily skewed or not normally distributed? Decision trees are nonparametric, meaning we don't make assumptions about how our data or errors are distributed.

### 3. Easy to interpret.

The output of a decision tree is easy to interpet and thus relatable to non-technical people. (We'll talk about `feature_importance` later.)

### 4. Speed.

Decision trees are fit very quickly!

> **Protip**
>
> Consider creating a benchmark using a decision tree to understand how one might behave on its own before using a more complicated method.  Do your best to understand how a simple model behaves on a set of data, before using a more complex model. This is a great practice to get into.

## What are the downsides of decision trees?


### 1. Decision trees can very easily overfit.
Decision trees often suffer from high error due to variance, so we need to take special care to avoid this. (Luckily, there are lots of algorithms designed to do exactly this!)

### 2. Decision trees are locally optimal.
Because we're making the best decision at each node (greedy), we might end up with a worse solution in the long run.

### 3. Decision trees don't work well with unbalanced data.
We often will bias our results toward the majority class. We need to take steps to avoid this as well! (Check out the `class_weight` parameter if you're interested.)